Go to http://skyserver.sdss.org/dr17/SearchTools/sql 
![apogee query page](<images/sdss-query-page.png>)

The query
```
-- it was limited to 500k rows only by server

SELECT TOP 500000
s.vhelio_avg AS rv_apogee,s.vscatter AS e_rv_apogee,
a.teff AS teff_apogee, a.teff_err AS e_teff_apogee, a.logg AS logg_apogee, 
a.logg_err AS e_logg_apogee, a.m_h AS mh_apogee, a.m_h_err AS e_mh_apogee,
a.fe_h AS feh_apogee, a.fe_h_err AS e_feh_apogee, 
a.alpha_m AS alpham_apogee, a.alpha_m_err AS e_alpham_apogee
FROM apogeeStar s
JOIN aspcapStar a on a.apstar_id = s.apstar_id
WHERE (a.aspcapflag & dbo.fApogeeAspcapFlag('STAR_BAD')) = 0 and
(s.apogee_target1 &
(dbo.fApogeeTarget1('APOGEE_SHORT')+
dbo.fApogeeTarget1('APOGEE_INTERMEDIATE')+
dbo.fApogeeTarget1('APOGEE_LONG'))) != 0
AND s.ra BETWEEN 0 AND 60
```

download as csv, because it has header at the first row

In [ ]:
import vaex
from glob import glob
import pandas as pd

In [ ]:
# let say the file name is
file = 'apogee.csv'

In [ ]:
# skip first row
df_pandas = pd.read_csv(file, skiprows=1)

In [ ]:
# convert to vaex
df = vaex.from_pandas(df_pandas)

In [ ]:
# mask outliers
masking = ((df.e_teff_apogee > 0)*(df.e_teff_apogee < 200)*
           (df.e_rv_apogee > 0)*(df.e_rv_apogee < 500)*
           (df.e_mh_apogee > 0)*(df.e_mh_apogee < 0.1)*
           (df.e_feh_apogee > 0)*(df.e_feh_apogee < 0.1)*
           (df.e_alpham_apogee > 0)*(df.e_alpham_apogee < 0.1))
df_filtered = df[masking]
df_filtered = df_filtered.extract()
df_filtered

In [ ]:
# convert to hdf5
df_filtered.export("apogee.hdf5", progress=True)